In [1]:
import sys
import json
import requests
import time
import argparse
import pandas as pd
import numpy as np
# import pymongo

import random

import boto3
import boto.s3.connection
from boto.s3.key import Key
from Airbnb_Scraping_tools import *
from urllib.parse import urlparse
from urllib.request import urlretrieve 
import os

In [2]:
%load_ext autoreload
%autoreload 2

# Load in data

In [3]:
s3_path = 'https://s3-us-west-2.amazonaws.com/chen-gal-test/AirbnbData/'

In [4]:
s3_path

'https://s3-us-west-2.amazonaws.com/chen-gal-test/AirbnbData/'

In [5]:
folder = 'Seattle-washington-US'
file_tmp = pd.read_csv(s3_path+folder+'/listings.csv.gz',compression = 'gzip')
print('Total number of apartments in ' + folder +" is: {}".format(len(file_tmp)))
file_tmp.describe()

Total number of apartments in Seattle-washington-US is: 31253


/home/ubuntu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,scrape_id,host_id,host_acceptance_rate,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,...,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month
count,3.125300e+04,3.125300e+04,3.125300e+04,0.0,31239.000000,31239.000000,0.0,31253.000000,31253.000000,31253.000000,...,31253.000000,23726.000000,23689.000000,23688.000000,23661.000000,23690.000000,23660.000000,23648.000000,31253.000000,24030.000000
mean,1.085829e+07,2.017050e+13,3.632524e+07,NaN,5.171132,5.171132,NaN,34.056220,-118.334252,3.425751,...,20.842543,94.214533,9.612394,9.416456,9.770297,9.768974,9.612933,9.485411,3.546572,1.857360
std,5.527237e+06,3.531306e+00,3.454658e+07,NaN,24.143834,24.143834,NaN,0.101912,0.153783,2.481926,...,38.106641,8.040304,0.789394,0.954680,0.652473,0.658068,0.735289,0.834739,5.551354,2.022831
min,1.090000e+02,2.017050e+13,5.900000e+01,NaN,0.000000,0.000000,NaN,33.338905,-118.911026,1.000000,...,0.000000,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.020000
25%,6.262191e+06,2.017050e+13,7.441819e+06,NaN,1.000000,1.000000,NaN,34.013997,-118.431313,2.000000,...,1.000000,92.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,1.000000,0.410000
50%,1.216199e+07,2.017050e+13,2.471631e+07,NaN,2.000000,2.000000,NaN,34.066607,-118.349465,2.000000,...,6.000000,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000,1.120000
75%,1.570678e+07,2.017050e+13,5.623875e+07,NaN,4.000000,4.000000,NaN,34.103960,-118.267693,4.000000,...,23.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,3.000000,2.730000
max,1.852404e+07,2.017050e+13,1.285513e+08,NaN,855.000000,855.000000,NaN,34.742088,-117.655471,16.000000,...,605.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,49.000000,90.000000


# Open S3 bucket

In [6]:
bucket_name = "chen-gal-test"

In [7]:
s3 = boto3.client("s3")

remote_pathname_l1 = 'AirbnbData/'
remote_pathname_l2 = folder
remote_pathname = remote_pathname_l1 + remote_pathname_l2

# s3.put_object(Body=file , Bucket=bucket_name, Key=remote_pathname)


In [8]:
remote_pathname

'AirbnbData/Seattle-washington-US'

In [9]:
logfile_name = '../DataSet/'+'logfile_'+folder+'.txt'
print(f'Logfile name is {logfile_name}')

Logfile name is ../DataSet/logfile_Seattle-washington-US.txt


In [10]:
if not os.path.exists('../DataSet/'):
    os.makedirs('..DataSet/')

# Downloading

In [11]:

apt_infos = {}
apt_jses = {}

In [ ]:
for i in range(18001,len(file_tmp)):
# for i in range(1):
    print(i)
    url = file_tmp.loc[i,'listing_url']
    apt_id = file_tmp.loc[i,'id']
    apt_name = file_tmp.loc[i,'name']
    with open(logfile_name,'a') as f:
        f.write(f'---------------------\nApartment{i} | {apt_id} | {url} \n')         
    try:
        return_parsed = parse_each_webpage(url)
    except CannotOpenUrl:        
        with open(logfile_name,'a') as f:
            f.writelines('Cannot open '+ str(i)+ ' url.\t' + 'Id: '+ str(apt_id) + ' |  Url: '+ url)                        
        print('Failed to get access to URL, sleep for 20 secs ')
        time.sleep(20)
        return_parsed = parse_each_webpage(url)
    finally:
        scrape_info,ori_json = return_parsed[0],return_parsed[1]
        if len(scrape_info) == 0:
            with open(logfile_name,'a') as f:
                f.writelines('Failed to scrape the url, simply save the json file\n')
        else:
            with open(logfile_name,'a') as f:
                f.writelines('Successfully scraped the url, save both files\n')
            
        dict_apt = {'apt_name':apt_name,'url' : url, 'info': scrape_info}
#         apt_jses.update({str(apt_id):ori_json})
        apt_infos.update({str(apt_id):dict_apt})
        with open(logfile_name,'a') as f:
                f.writelines(f'Finished scraping {i} apartment! ID:{apt_id}\n')
                
#         if i % 75 == 0 and i != 0:            
#             js_filename = remote_pathname +"/jsons/ori_json_"+str(i)            
#             s3.put_object(Body=json.dumps(apt_jses) , Bucket=bucket_name, Key=js_filename)
            
#             print("Wrote  down"+str(i)+"json file")
#             with open(logfile_name,'a') as f:
#                 f.writelines(f'Saved json file {js_filename}!\n') 
        if i % 2000 == 0 and i != 0:            
            wsdata_filename = remote_pathname +"/ws_data/scraped_partial_data_"+str(i)            
            s3.put_object(Body=json.dumps(apt_infos) , Bucket=bucket_name, Key=wsdata_filename)
            
            print("Wrote  down"+str(i)+"ws_data")
            with open(logfile_name,'a') as f:
                f.writelines(f'Saved ws_data file {wsdata_filename}!\n')  
            apt_infos = {}
            
        time.sleep(random.random()+2)
        
#upload log_file to s3
logfile_s3_filename = remote_pathname+'/ws_data/log_file.txt'
s3.upload_file(Bucket = bucket_name,Filename=logfile_name,Key=logfile_s3_filename)

scraped_filename = remote_pathname +"/ws_data/webscrapted_last_piece.json"

s3.put_object(Body=json.dumps(apt_infos) , Bucket=bucket_name, Key=scraped_filename)

18001
request_url:https://www.airbnb.com/rooms/11609642
18002
request_url:https://www.airbnb.com/rooms/17875513
18003
request_url:https://www.airbnb.com/rooms/15662066
18004
request_url:https://www.airbnb.com/rooms/16579301
18005
request_url:https://www.airbnb.com/rooms/3405964
18006
request_url:https://www.airbnb.com/rooms/12811708
18007
request_url:https://www.airbnb.com/rooms/14611204
18008
request_url:https://www.airbnb.com/rooms/5165050
18009
request_url:https://www.airbnb.com/rooms/16629616
18010
request_url:https://www.airbnb.com/rooms/9943465
18011
request_url:https://www.airbnb.com/rooms/16823268
18012
request_url:https://www.airbnb.com/rooms/7054856
18013
request_url:https://www.airbnb.com/rooms/15835794
18014
request_url:https://www.airbnb.com/rooms/2007610
18015
request_url:https://www.airbnb.com/rooms/9936729
18016
request_url:https://www.airbnb.com/rooms/16405296
18017
request_url:https://www.airbnb.com/rooms/11444652
18018
request_url:https://www.airbnb.com/rooms/1576871

18149
request_url:https://www.airbnb.com/rooms/17530899
18150
request_url:https://www.airbnb.com/rooms/14256305
18151
request_url:https://www.airbnb.com/rooms/7650792
18152
request_url:https://www.airbnb.com/rooms/14400452
18153
request_url:https://www.airbnb.com/rooms/12836881
18154
request_url:https://www.airbnb.com/rooms/14119171
18155
request_url:https://www.airbnb.com/rooms/18488616
18156
request_url:https://www.airbnb.com/rooms/15405600
18157
request_url:https://www.airbnb.com/rooms/17256724
18158
request_url:https://www.airbnb.com/rooms/5022210
18159
request_url:https://www.airbnb.com/rooms/13978113
18160
request_url:https://www.airbnb.com/rooms/18497780
18161
request_url:https://www.airbnb.com/rooms/14337214
18162
request_url:https://www.airbnb.com/rooms/15269312
18163
request_url:https://www.airbnb.com/rooms/12055710
18164
request_url:https://www.airbnb.com/rooms/17811281
18165
request_url:https://www.airbnb.com/rooms/16301209
18166
request_url:https://www.airbnb.com/rooms/131

18293
request_url:https://www.airbnb.com/rooms/17535604
18294
request_url:https://www.airbnb.com/rooms/11848384
18295
request_url:https://www.airbnb.com/rooms/12546289
18296
request_url:https://www.airbnb.com/rooms/1291014
18297
request_url:https://www.airbnb.com/rooms/18231512
18298
request_url:https://www.airbnb.com/rooms/16592324
18299
request_url:https://www.airbnb.com/rooms/13315146
18300
request_url:https://www.airbnb.com/rooms/7179635
18301
request_url:https://www.airbnb.com/rooms/2919690
18302
request_url:https://www.airbnb.com/rooms/745011
18303
request_url:https://www.airbnb.com/rooms/17622195
18304
request_url:https://www.airbnb.com/rooms/4794766
18305
request_url:https://www.airbnb.com/rooms/3116534
18306
request_url:https://www.airbnb.com/rooms/4720478
18307
request_url:https://www.airbnb.com/rooms/18274861
18308
request_url:https://www.airbnb.com/rooms/6132159
18309
request_url:https://www.airbnb.com/rooms/14031680
18310
request_url:https://www.airbnb.com/rooms/9512793
18

18441
request_url:https://www.airbnb.com/rooms/15299255
18442
request_url:https://www.airbnb.com/rooms/6845430
18443
request_url:https://www.airbnb.com/rooms/1012052
18444
request_url:https://www.airbnb.com/rooms/17557565
18445
request_url:https://www.airbnb.com/rooms/16543714
18446
request_url:https://www.airbnb.com/rooms/14315015
18447
request_url:https://www.airbnb.com/rooms/15363304
18448
request_url:https://www.airbnb.com/rooms/3042436
18449
request_url:https://www.airbnb.com/rooms/14651778
18450
request_url:https://www.airbnb.com/rooms/5930112
18451
request_url:https://www.airbnb.com/rooms/17720950
18452
request_url:https://www.airbnb.com/rooms/9529246
18453
request_url:https://www.airbnb.com/rooms/1665388
18454
request_url:https://www.airbnb.com/rooms/13329701
18455
request_url:https://www.airbnb.com/rooms/6578910
18456
request_url:https://www.airbnb.com/rooms/13264910
18457
request_url:https://www.airbnb.com/rooms/7956484
18458
request_url:https://www.airbnb.com/rooms/15099822


18555
request_url:https://www.airbnb.com/rooms/2898128
18556
request_url:https://www.airbnb.com/rooms/16004815
18557
request_url:https://www.airbnb.com/rooms/12871413
18558
request_url:https://www.airbnb.com/rooms/6436790
18559
request_url:https://www.airbnb.com/rooms/10640865
18560
request_url:https://www.airbnb.com/rooms/14430980
18561
request_url:https://www.airbnb.com/rooms/9974487
18562
request_url:https://www.airbnb.com/rooms/13644370
18563
request_url:https://www.airbnb.com/rooms/9351948
18564
request_url:https://www.airbnb.com/rooms/16724525
18565
request_url:https://www.airbnb.com/rooms/12989983
18566
request_url:https://www.airbnb.com/rooms/14300168
18567
request_url:https://www.airbnb.com/rooms/11743628
18568
request_url:https://www.airbnb.com/rooms/14234049
18569
request_url:https://www.airbnb.com/rooms/6237807
18570
request_url:https://www.airbnb.com/rooms/6630775
18571
request_url:https://www.airbnb.com/rooms/15871292
18572
request_url:https://www.airbnb.com/rooms/1515038

...

20637
request_url:https://www.airbnb.com/rooms/15873376
20638
request_url:https://www.airbnb.com/rooms/11416899
20639
request_url:https://www.airbnb.com/rooms/9645964
20640
request_url:https://www.airbnb.com/rooms/4795286
20641
request_url:https://www.airbnb.com/rooms/15085952
20642
request_url:https://www.airbnb.com/rooms/272637
20643
request_url:https://www.airbnb.com/rooms/18294906
20644
request_url:https://www.airbnb.com/rooms/17902755
20645
request_url:https://www.airbnb.com/rooms/8596057
20646
request_url:https://www.airbnb.com/rooms/13117863
20647
request_url:https://www.airbnb.com/rooms/14231889
20648
request_url:https://www.airbnb.com/rooms/14231092
20649
request_url:https://www.airbnb.com/rooms/13043291
20650
request_url:https://www.airbnb.com/rooms/17777050
20651
request_url:https://www.airbnb.com/rooms/7479585
20652
request_url:https://www.airbnb.com/rooms/17849041
20653
request_url:https://www.airbnb.com/rooms/6627511
20654
request_url:https://www.airbnb.com/rooms/10541934

20751
request_url:https://www.airbnb.com/rooms/17346308
20752
request_url:https://www.airbnb.com/rooms/8431794
20753
request_url:https://www.airbnb.com/rooms/10724597
20754
request_url:https://www.airbnb.com/rooms/3842716
20755
request_url:https://www.airbnb.com/rooms/14590898
20756
request_url:https://www.airbnb.com/rooms/14786910
20757
request_url:https://www.airbnb.com/rooms/16586997
20758
request_url:https://www.airbnb.com/rooms/14255429
20759
request_url:https://www.airbnb.com/rooms/2127823
20760
request_url:https://www.airbnb.com/rooms/11761606
20761
request_url:https://www.airbnb.com/rooms/9532329
20762
request_url:https://www.airbnb.com/rooms/17657885
20763
request_url:https://www.airbnb.com/rooms/1797650
20764
request_url:https://www.airbnb.com/rooms/16869373
20765
request_url:https://www.airbnb.com/rooms/9136094
20766
request_url:https://www.airbnb.com/rooms/9788672
20767
request_url:https://www.airbnb.com/rooms/18240432
20768
request_url:https://www.airbnb.com/rooms/11227337

20899
request_url:https://www.airbnb.com/rooms/15328323
20900
request_url:https://www.airbnb.com/rooms/11463960
20901
request_url:https://www.airbnb.com/rooms/8719388
20902
request_url:https://www.airbnb.com/rooms/16976369
20903
request_url:https://www.airbnb.com/rooms/12805905
20904
request_url:https://www.airbnb.com/rooms/16573467
20905
request_url:https://www.airbnb.com/rooms/18041799
20906
request_url:https://www.airbnb.com/rooms/18208907
20907
request_url:https://www.airbnb.com/rooms/5727323
20908
request_url:https://www.airbnb.com/rooms/14593569
20909
request_url:https://www.airbnb.com/rooms/13393782
20910
request_url:https://www.airbnb.com/rooms/6184764
20911
request_url:https://www.airbnb.com/rooms/16810005
20912
request_url:https://www.airbnb.com/rooms/16188579
20913
request_url:https://www.airbnb.com/rooms/4708306
20914
request_url:https://www.airbnb.com/rooms/16381093
20915
request_url:https://www.airbnb.com/rooms/13083865
20916
request_url:https://www.airbnb.com/rooms/14472

21047
request_url:https://www.airbnb.com/rooms/882329
21048
request_url:https://www.airbnb.com/rooms/1346324
21049
request_url:https://www.airbnb.com/rooms/7665091
21050
request_url:https://www.airbnb.com/rooms/2815278
21051
request_url:https://www.airbnb.com/rooms/14808336
21052
request_url:https://www.airbnb.com/rooms/16053498
21053
request_url:https://www.airbnb.com/rooms/13497593
21054
request_url:https://www.airbnb.com/rooms/12285683
21055
request_url:https://www.airbnb.com/rooms/17560743
21056
request_url:https://www.airbnb.com/rooms/1045627
21057
request_url:https://www.airbnb.com/rooms/12370520
21058
request_url:https://www.airbnb.com/rooms/8366986
21059
request_url:https://www.airbnb.com/rooms/15903572
21060
request_url:https://www.airbnb.com/rooms/12889395
21061
request_url:https://www.airbnb.com/rooms/11709675
21062
request_url:https://www.airbnb.com/rooms/7777537
21063
request_url:https://www.airbnb.com/rooms/13282382
21064
request_url:https://www.airbnb.com/rooms/15398296


21195
request_url:https://www.airbnb.com/rooms/7054687
21196
request_url:https://www.airbnb.com/rooms/18505317
21197
request_url:https://www.airbnb.com/rooms/16633969
21198
request_url:https://www.airbnb.com/rooms/6214893
21199
request_url:https://www.airbnb.com/rooms/14965867
21200
request_url:https://www.airbnb.com/rooms/8673111
21201
request_url:https://www.airbnb.com/rooms/17814532
21202
request_url:https://www.airbnb.com/rooms/6716228
21203
request_url:https://www.airbnb.com/rooms/16013545
21204
request_url:https://www.airbnb.com/rooms/15131408
21205
request_url:https://www.airbnb.com/rooms/16274753
21206
request_url:https://www.airbnb.com/rooms/6706330
21207
request_url:https://www.airbnb.com/rooms/10179859
21208
request_url:https://www.airbnb.com/rooms/14045579
21209
request_url:https://www.airbnb.com/rooms/7819664
21210
request_url:https://www.airbnb.com/rooms/14373076
21211
request_url:https://www.airbnb.com/rooms/15225003
21212
request_url:https://www.airbnb.com/rooms/9064506

In [15]:
logfile_s3_filename = remote_pathname+'/ws_data/log_file.txt'
s3.upload_file(Bucket = bucket_name,Filename=logfile_name,Key=logfile_s3_filename)

In [102]:
with open(logfile_name,'a') as f:
    f.writelines(f'Saved json file {js_filename}!\n')                            
jses = {}

## Save the result

### save scrapted data

In [16]:
scraped_filename = remote_pathname +"/ws_data/webscrapted.json"

s3.put_object(Body=json.dumps(apt_infos) , Bucket=bucket_name, Key=scraped_filename)

{'ETag': '"e5be8054bac84d9f037ecc369719eb7d"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Wed, 02 May 2018 04:27:30 GMT',
   'etag': '"e5be8054bac84d9f037ecc369719eb7d"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'rjbHFiw1U3qRRNSXOpal38U9mq2s7b7uRxZNKlUeZvrbBGwWPBGXSUeC4HBQ13A30POoEh8E8c8=',
   'x-amz-request-id': 'FA72BE06AE13EEC8'},
  'HTTPStatusCode': 200,
  'HostId': 'rjbHFiw1U3qRRNSXOpal38U9mq2s7b7uRxZNKlUeZvrbBGwWPBGXSUeC4HBQ13A30POoEh8E8c8=',
  'RequestId': 'FA72BE06AE13EEC8',
  'RetryAttempts': 0}}

### save json data

In [88]:
# last_piece_js_filename = remote_pathname+"/jsons/ori_json_lastpiece.json"

# s3.put_object(Body=json.dumps(apt_jses) , Bucket=bucket_name, Key=last_piece_js_filename)


{'ETag': '"7b336feb550bfad1dc2c55de6ec8a93d"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Mon, 30 Apr 2018 01:22:39 GMT',
   'etag': '"7b336feb550bfad1dc2c55de6ec8a93d"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'S7BG7n3tOOJQXdwejYwoPOwpqzfEF9NGChTaAzo/bBZ5ZmmQJvaioLk2y3mYnDQtAsnUZ52XF/k=',
   'x-amz-request-id': 'BB474E35A45B7DAC'},
  'HTTPStatusCode': 200,
  'HostId': 'S7BG7n3tOOJQXdwejYwoPOwpqzfEF9NGChTaAzo/bBZ5ZmmQJvaioLk2y3mYnDQtAsnUZ52XF/k=',
  'RequestId': 'BB474E35A45B7DAC',
  'RetryAttempts': 0}}

## Load json data from s3

In [11]:
scraped_filename = 'AirbnbData/Boston-Massachusetts-US/ws_data/scraped_partial_data_3600'

response = s3.get_object(Bucket=bucket_name,
                         Key=scraped_filename)

In [19]:
response

{'AcceptRanges': 'bytes',
 'Body': <botocore.response.StreamingBody at 0x7fee21cc0d30>,
 'ContentLength': 5059688,
 'ContentType': 'binary/octet-stream',
 'ETag': '"3aa75aea13d6b1d083cea56b1332defc"',
 'LastModified': datetime.datetime(2018, 4, 30, 14, 25, 10, tzinfo=tzutc()),
 'Metadata': {},
 'ResponseMetadata': {'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '5059688',
   'content-type': 'binary/octet-stream',
   'date': 'Tue, 01 May 2018 05:46:32 GMT',
   'etag': '"3aa75aea13d6b1d083cea56b1332defc"',
   'last-modified': 'Mon, 30 Apr 2018 14:25:10 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'VgQXishVBTFtclbB8DJ3hcIxNKV9hLHaJQwz+0jizaRHyz3WHejmgO/NCmR2NeXNjU4CMeGdTJo=',
   'x-amz-request-id': 'D35DCF8A8B8701D1'},
  'HTTPStatusCode': 200,
  'HostId': 'VgQXishVBTFtclbB8DJ3hcIxNKV9hLHaJQwz+0jizaRHyz3WHejmgO/NCmR2NeXNjU4CMeGdTJo=',
  'RequestId': 'D35DCF8A8B8701D1',
  'RetryAttempts': 0}}

In [12]:
scraped_data = json.loads(response['Body'].read())

In [13]:
apt_infos = scraped_data

In [14]:
len(apt_infos)

3600